## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,64.60,35,0,8.68,clear sky
1,1,Nanakuli,US,21.3906,-158.1547,84.15,56,40,18.12,scattered clouds
2,2,Yumen,CN,40.2833,97.2000,58.30,23,78,12.15,broken clouds
3,3,Souillac,MU,-20.5167,57.5167,71.37,84,58,11.56,light rain
4,4,Hobart,AU,-42.8794,147.3294,52.27,82,75,1.01,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                420
City                   420
Country                416
Lat                    420
Lng                    420
Max Temp               420
Humidity               420
Cloudiness             420
Wind Speed             420
Current Description    420
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                416
City                   416
Country                416
Lat                    416
Lng                    416
Max Temp               416
Humidity               416
Cloudiness             416
Wind Speed             416
Current Description    416
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,64.60,clear sky,-33.5906,26.8910,
1,Nanakuli,US,84.15,scattered clouds,21.3906,-158.1547,
3,Souillac,MU,71.37,light rain,-20.5167,57.5167,
5,Pangai,TO,77.61,clear sky,-19.8000,-174.3500,
7,Tautira,PF,86.02,scattered clouds,-17.7333,-149.1500,
10,Salinopolis,BR,78.15,scattered clouds,-0.6136,-47.3561,
12,Praia,CV,73.94,broken clouds,14.9215,-23.5087,
13,Hermanus,ZA,65.89,clear sky,-34.4187,19.2345,
14,Axim,GH,79.29,few clouds,4.8699,-2.2405,
15,Bad Freienwalde,DE,69.30,scattered clouds,52.7873,14.0304,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found for city... skipping Lame City.")

Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lame City.
Hotel not found for city... skipping Lam

In [9]:
# 7. Drop the rows where there is no Hotel Name.
# The rows with no hotel names are not being recognized as empty, replaced the empty value with np.nan.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace("",np.nan)
# Then dropna will recognize them as an empty hotel name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   387
Country                387
Max Temp               387
Current Description    387
Lat                    387
Lng                    387
Hotel Name             387
dtype: int64

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,64.60,clear sky,-33.5906,26.8910,The Halyards Hotel
1,Nanakuli,US,84.15,scattered clouds,21.3906,-158.1547,Camp Pālehua
3,Souillac,MU,71.37,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
5,Pangai,TO,77.61,clear sky,-19.8000,-174.3500,Ha'apai Beach Resort
7,Tautira,PF,86.02,scattered clouds,-17.7333,-149.1500,Pension Oaoa
...,...,...,...,...,...,...,...
695,Balabac,PH,82.53,overcast clouds,7.9868,117.0645,Sing and Swing Lodge
697,Les Clayes-Sous-Bois,FR,64.08,clear sky,48.8221,1.9868,Hôtel Kyriad Plaisir
698,Buchanan,LR,76.96,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
699,Locri,IT,74.32,overcast clouds,38.2414,16.2623,Grand Hotel President


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# format of the info box on the map figure.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))